# Measuring Prompt Energy in LLMs

**Research tool for analyzing prompt characteristics vs. energy consumption across LLM models**

## Setup:
1. Install: `pip install -r requirements.txt`
2. Add API keys to `.env`:
   ```
   OPENAI_API_KEY=your_key
   GROQ_API_KEY=your_key  
   MISTRAL_API_KEY=your_key
   ```
3. Run cells to collect data

## Measures:
- Energy consumption (Wh) via CodeCarbon
- Token usage and response timing
- Time to first token
- Performance across OpenAI, Groq, Mistral models

In [68]:
# Initialize dependencies
import pandas as pd, openai, os, json, time, requests
from dotenv import load_dotenv
from codecarbon import EmissionsTracker

load_dotenv()

True

In [69]:
# Load data and results
data = pd.read_json("data/sample.json")
if 'processed' not in data.columns:
    data['processed'] = 0
    data.to_json("data/sample.json", orient='records', indent=2)

try:
    with open("data/energy.json", "r") as f:
        results = json.load(f)
except FileNotFoundError:
    results = []

In [70]:
# Setup API clients
load_dotenv(override=True)
clients, models_to_test = {}, []

if openai_key := os.getenv("OPENAI_API_KEY"):
    clients['openai'] = openai.OpenAI(api_key=openai_key)
    models_to_test.append("openai")

if groq_key := os.getenv("GROQ_API_KEY"):
    clients['llama'] = groq_key
    models_to_test.append("llama")

if mistral_key := os.getenv("MISTRAL_API_KEY"):
    clients['mistral'] = mistral_key
    models_to_test.append("mistral")


In [71]:
# Track performance with energy measurement
def track_performance(model_name, prompt, api_call_func):
    tracker = EmissionsTracker(log_level="ERROR", save_to_file=False)
    tracker.start()
    
    start_time = time.time()
    response_data, time_to_first_token = api_call_func(prompt)
    duration = time.time() - start_time
    
    energy_consumed_wh = tracker.stop() * 1000
    
    usage = response_data.get("usage", {})
    prompt_tokens = usage.get("prompt_tokens", usage.get("input_tokens", 0))
    completion_tokens = usage.get("completion_tokens", usage.get("output_tokens", 0))
    total_tokens = usage.get("total_tokens", prompt_tokens + completion_tokens)
    
    content = response_data.get("choices", [{}])[0].get("message", {}).get("content", "")
    
    return {
        "prompt": prompt,
        "model": response_data.get("model", model_name),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "duration": round(duration, 2),
        "time_to_first_token": round(time_to_first_token, 3),
        "prompt_tokens": prompt_tokens,
        "completion_tokens": completion_tokens,
        "total_tokens": total_tokens,
        "tokens_per_second": round(total_tokens / duration, 1) if duration > 0 else 0,
        "energy_consumed_wh": round(energy_consumed_wh, 4),
        "response": content
    }


In [72]:
# API call functions
def call_openai(prompt):
    start_time = time.time()
    response = clients['openai'].chat.completions.create(
        model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}], 
        max_tokens=50, temperature=0.3
    )
    
    return {
        "usage": {
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        },
        "choices": [{"message": {"content": choice.message.content}} for choice in response.choices],
        "model": response.model
    }, time.time() - start_time

def call_llama(prompt):
    start_time = time.time()
    response = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={"Authorization": f"Bearer {clients['llama']}"},
        json={"model": "llama-3.1-8b-instant", "messages": [{"role": "user", "content": prompt}], 
              "max_tokens": 50, "temperature": 0.3}
    )
    if response.status_code != 200:
        raise Exception(f"API Error: {response.status_code}")
    
    return response.json(), time.time() - start_time

def call_mistral(prompt):
    start_time = time.time()
    response = requests.post(
        "https://api.mistral.ai/v1/chat/completions",
        headers={"Authorization": f"Bearer {clients['mistral']}"},
        json={"model": "mistral-large-latest", "messages": [{"role": "user", "content": prompt}], 
              "max_tokens": 50, "temperature": 0.3}
    )
    if response.status_code != 200:
        raise Exception(f"API Error: {response.status_code}")
    resp_json = response.json()
    if "error" in resp_json:
        raise Exception(f"API Error: {resp_json['error']}")
    
    return resp_json, time.time() - start_time


In [73]:
# Data collection with progress tracking
def run_performance_tests(max_prompts=2):
    global data, results
    
    unprocessed = data[data['processed'] == 0].head(max_prompts)
    if len(unprocessed) == 0:
        print("All prompts processed")
        return
    
    print(f"Processing {len(unprocessed)} prompts with {len(models_to_test)} models")
    
    new_results = []
    api_calls = {"openai": call_openai, "llama": call_llama, "mistral": call_mistral}
    
    for idx, row in unprocessed.iterrows():
        prompt_text = row['prompt_text']
        prompt_num = idx + 1
        
        for model_name in models_to_test:
            print(f"\rProcessing prompt {prompt_num} with {model_name}...", end="", flush=True)
            result = track_performance(model_name, prompt_text, api_calls[model_name])
            new_results.append(result)
        
        data.loc[idx, 'processed'] = 1
    
    data.to_json("data/sample.json", orient='records', indent=2)
    all_results = results + new_results
    with open("data/energy.json", "w") as f:
        json.dump(all_results, f, indent=2)
    
    total_tokens = sum(r['total_tokens'] for r in new_results)
    print(f"\rCompleted: {len(new_results)} calls, {total_tokens:,} total tokens")
    return all_results

if models_to_test:
    all_results = run_performance_tests(max_prompts=2)


Processing 2 prompts with 3 models
Completed: 6 calls, 558 total tokens


In [67]:
# Reset processed status
data['processed'] = 0
data.to_json("data/sample.json", orient='records', indent=2)
